# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1

In [1]:
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
d=pd.read_html(url)

In [3]:
df=d[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
df1=df.loc[df['Borough']!='Not assigned']

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [5]:
df1.loc[df1['Neighbourhood']=='Not assigned','Neighbourhood'] = df1['Borough']
df1.loc[df1['Neighbourhood']=="Queen's Park"]

/home/ishwar/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/ishwar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Queen's Park


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [6]:
df2=df1.groupby('Postcode').agg({'Borough': pd.Series.unique ,'Neighbourhood': lambda x: ','.join(x)})
df2.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
df2.shape

(103, 2)

## Part 2

Reading into a pandas dataframe using this link to the csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [9]:
csv_url='http://cocl.us/Geospatial_data'
geo=pd.read_csv(csv_url)
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging both the dataframes so that the new dataframe has the following columns:
Postcode, Borough, Neighbourhood, Latitude, Longitude

In [10]:
geo.rename(columns={'Postal Code':'Postcode'},inplace=True)
df_merge_col = pd.merge(df2, geo, on='Postcode')
toronto=df_merge_col
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

Importing necessary libraries:

In [12]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ishwar/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |             main           3 KB
    altair-3.2.0               |           py37_0         773 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.7.11               |           py37_0         3.0 MB  conda-forge
    conda-package-handling-1.4.1|           py37_0         262 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forg

 I'll be working with only boroughs that contain the word Toronto 

In [18]:
toronto_data = toronto[toronto['Borough'].str.contains(pat='Toronto')].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Analyzing the neighbourhoods:

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_data[['Borough']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_data['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Central Toronto,Downtown Toronto,East Toronto,West Toronto
0,The Beaches,0,0,1,0
1,"The Danforth West,Riverdale",0,0,1,0
2,"The Beaches West,India Bazaar",0,0,1,0
3,Studio District,0,0,1,0
4,Lawrence Park,1,0,0,0


In [24]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Central Toronto,Downtown Toronto,East Toronto,West Toronto
0,"Adelaide,King,Richmond",0,1,0,0
1,Berczy Park,0,1,0,0
2,"Brockton,Exhibition Place,Parkdale Village",0,0,0,1
3,Business Reply Mail Processing Centre 969 Eastern,0,0,1,0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0,1,0,0
5,"Cabbagetown,St. James Town",0,1,0,0
6,Central Bay Street,0,1,0,0
7,"Chinatown,Grange Park,Kensington Market",0,1,0,0
8,Christie,0,1,0,0
9,Church and Wellesley,0,1,0,0


Using k means clustering method to analyze the data and creating clusters:

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/home/ishwar/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 2, 3, 0, 0, 0, 0, 0, 0], dtype=int32)

In [30]:
toronto_data_sorted.head()

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,0,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


## Visualizing the clusters using a map

In [34]:
# create map
latitude=43.728020
longitude=-79.388790
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters